In [2]:
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]

  Cloning https://github.com/neuml/txtai to /tmp/pip-install-371z53fu/txtai_b7f6476039fb495db013f6374c1221b5
  Running command git clone -q https://github.com/neuml/txtai /tmp/pip-install-371z53fu/txtai_b7f6476039fb495db013f6374c1221b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [3]:
!pip install jsonlines
import jsonlines
!pip install --no-cache-dir transformers sentencepiece
!pip install langdetect
from txtai.pipeline import Translation
from langdetect import detect
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import json
import glob
import nltk
nltk.download('punkt')
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No GPU available, using the CPU instead.


In [4]:
all_code = {'ar', 'bn', 'ja', 'fi', 'ru', 'ko', 'te'}
translate = Translation()
#translation = translate("This is a test translation into Spanish", "es")
#print(translation)

# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_ans_across_lang_nofilter.json") as f:
#   results = json.load(f)

with open("/content/drive/MyDrive/UMD Academics/Spring 2022/CMSC848Q Selected Topics in Information Processing; How and Why Artificial Intelligence Answers Questions /CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_results_100_long_ctxlang_across_lang_nofilter.json") as f:
  results = json.load(f)

In [5]:
len(results)

5001

In [6]:
import torch
print(torch.__version__)

1.11.0+cu113


In [ ]:
# {'ctx_lang': 'en',
#    'has_answer': True,
#    'id': '17763030',
#    'score': '39.482452',
#    'text': "resulting genocides and the 1918 influenza pandemic caused another 50 to 100 million deaths worldwide. On 28 June 1914, Gavrilo Princip, a Bosnian Serb Yugoslav nationalist, assassinated the Austro-Hungarian heir Archduke Franz Ferdinand in Sarajevo, leading to the July Crisis. In response, on 23 July Austria-Hungary issued an ultimatum to Serbia. Serbia's reply failed to satisfy the Austrians, and the two moved to a war footing. A network of interlocking alliances enlarged the crisis from a bilateral issue in the Balkans to one involving most of Europe. By July 1914, the great powers of Europe were divided into two coalitions:",
#    'title': 'World War I'},
import gc
from tqdm import tqdm

all_data = []
final_data = []
# for datum in across_lang:
for idx, datum in tqdm(enumerate(results)):
  if idx <= 100*45:
    continue

  translated_language = []

  new_datum = []
  src_lang = datum['lang']
  qn = datum['question']
  answer = datum['answers']
  qn_en = translate(qn, 'en')
  ans_en = [translate(ans, 'en') for ans in answer]
  all_context_lang = set([context['ctx_lang'] for context in datum['ctxs'] if context['ctx_lang'] is not None])
  if len(all_context_lang) == 1:
    continue
  for context in datum['ctxs']:
    # if context['has_answer']:
      tar_lang = context['ctx_lang']
      if tar_lang in translated_language:
        continue

      if tar_lang is not None and "zh" in tar_lang:
        tar_lang = "zh"

      sample = {}
      if tar_lang == src_lang:
        for k in datum:
          if k != 'ctxs':
            sample[k] = datum[k]
        sample['context'] = context
        sample['orig_lang'] = src_lang
        sample['orig_qn'] = qn
        new_datum.append(sample)
        translated_language.append(tar_lang)
      # elif tar_lang in all_code: #tar_lang != src_lang
      else:
        try:
          translated_qn = translate(qn_en, tar_lang)
          translated_ans = [translate(ans, tar_lang) for ans in ans_en]
          translated_language.append(tar_lang)
        except:
          print("caught an exception, tar_lang is {}".format(tar_lang))
          continue
        for k in datum:
          if k != 'ctxs':
            sample[k] = datum[k]
        sample['question'] = translated_qn
        sample['answers'] = translated_ans
        sample['lang'] = tar_lang
        sample['context'] = context
        sample['orig_lang'] = src_lang
        sample['orig_qn'] = qn
        new_datum.append(sample)
  all_data.append(new_datum)
  # final_data.append(new_datum)

  break_down = 100
  if idx % break_down == 0 and idx != 0:
    with open("/content/drive/MyDrive/UMD Academics/Spring 2022/CMSC848Q Selected Topics in Information Processing; How and Why Artificial Intelligence Answers Questions /CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_top100_part{idx}.json".format(idx=idx//break_down), "w") as outfile:
      json.dump(all_data, outfile)
      print("Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_part{idx}.json".format(idx=idx//break_down))
      all_data = []
      torch.cuda.empty_cache()
      gc.collect()
  # break

# with open("/content/drive/MyDrive/CMSC848Q/multilingual-qa-bias/mDPR/xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json", "w") as outfile:
#       json.dump(final_data, outfile)
#       print("Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_final.json") 

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
4540it [10:09,  6.21s/it]

caught an exception, tar_lang is es


4561it [13:23, 15.65s/it]

Downloading:   0%|          | 0.00/795k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/810k [00:00<?, ?B/s]

...

4680it [40:10,  5.56s/it]

caught an exception, tar_lang is zh


4700it [45:42, 13.76s/it]

Wrote to xor_dev_dpr_retrieval_xlingual_samples_nofilter_part47.json


4701it [45:49, 11.71s/it]